In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
#from picamera import PiCamera
from time import sleep
from io import BytesIO
import sounddevice as sd

import PySimpleGUI as sg
import tkinter as tk
import PIL
from PIL import ImageTk
import cv2 
from time import sleep, strftime
#import io


In [2]:
print("Started")



#path = Path("/home/filip/Mushrooms")  ##for rpi

path = Path('/media/SSD/Mushrooms')  ##for desktop
path_train = path / 'train'

#camera=PiCamera()  ##for rpi
#camera.resolution = (1024,768)  ##for rpi



print("camera initialized...")
learn = load_learner(path_train,"squeezenet11_export.pkl");
learn.to_fp32();
print("model loaded...")



Started
camera initialized...


/home/filip/anaconda3/envs/fastai57/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/filip/anaconda3/envs/fastai57/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/filip/anaconda3/envs/fastai57/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve

model loaded...


In [ ]:
amplitude = 0.5
frequency = 1000
duration = 0.1
modulationf=1
start_idx = 0

def sinewave(t,amplitude = 0.5,frequency = 0,phase = 0):
    return amplitude*np.sin(phase+2 * np.pi * frequency * t)

def gate(t,duration):
    return (t<duration)

def periodicgate(t,frequency,delay = 0.25, stop=0.50):
    x=((t* frequency) % 1)/frequency
    return (x>=delay) & (x<=stop)

def sawtoothwave(t,frequency,delay=0):
    return (delay+  frequency * t) % 1 

samplerate = 16000

def callback(outdata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    global start_idx
    t = (start_idx + np.arange(frames)) / samplerate
    t = t.reshape(-1, 1)
    outdata[:] = sinewave(t,frequency=frequency) * periodicgate(t,modulationf,0,.05)
    start_idx += frames
    # use repeating function to repeat pulse (div or mod?).  Also can ramp up and down using modulating sine a lower freq




timer_running = True

#stream=BytesIO()  ##for rpi

#camera.capture(stream,format='jpeg',resize=(224,224))  ##for rpi
#stream.seek(0)  ##for rpi
#image=PIL.Image.open(stream).convert('RGB')  ##for rpi

camera = cv2.VideoCapture(0) ##for desktop
ret = camera.set(3,224)
ret = camera.set(4,224)

s, image = camera.read() ##for desktop
image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
#plt.imshow(image)
#imgtk = PIL.ImageTk.PhotoImage(image)

while True:
    with sd.OutputStream( channels=1, callback=callback,
                         samplerate=samplerate):
        if timer_running:


            ## rpi
            #stream=BytesIO()
            #camera.capture(stream,format='jpeg',resize=(224,224))
            #stream.seek(0)
            #image=PIL.Image.open(stream).convert('RGB')

            ##for desktop
            s, image = camera.read() ##for desktop
            image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
            img=Image(pil2tensor(image,np.float32).div_(255))
            #print("image captured and saved...")

            #image=np.empty((240,320,3),dtype=np.uint8)
            #camera.capture(image, 'rgb')
            #img=Image(tensor(image))
            pred_class,pred_idx,outputs = learn.predict(img)
            sleep(.1)
            prob=outputs[0].tolist()
            print(pred_class,prob)
            
            modulationf=prob*100
        
        
camera.release()




In [ ]:
camera.release()

In [ ]:
import PySimpleGUI as sg

layout = [  [sg.Text('Stopwatch', size=(20, 2), justification='center')],
            [sg.Text('', size=(10, 2), font=('Helvetica', 20), justification='center', key='_OUTPUT_')],
            [sg.T(' ' * 5), sg.Button('Start/Stop', focus=True), sg.Quit()]]

window = sg.Window('Stopwatch Timer', layout)
event, values = window.Read() 
window.Close()
print(event, values) 

In [ ]:
import PySimpleGUI as sg

layout = [  [sg.Text('Stopwatch', size=(20, 2), justification='center')],
            [sg.Text('', size=(10, 2), font=('Helvetica', 20), justification='center', key='_OUTPUT_')],
            [sg.T(' ' * 5), sg.Button('Start/Stop', focus=True), sg.Quit()]]

window = sg.Window('Stopwatch Timer', layout)

timer_running, i = True, 0

while True:        # Event Loop
    event, values = window.Read(timeout=10) # Please try and use as high of a timeout value as you can
    if event is None or event == 'Quit':    # if user closed the window using X or clicked Quit button
        break
    elif event == 'Start/Stop':
        timer_running = not timer_running
    if timer_running:
        window.Element('_OUTPUT_').Update('{:02d}:{:02d}.{:02d}'.format((i // 100) // 60, (i // 100) % 60, i % 100))
        i += 1

In [ ]:
## uses a ram io file to store image to get to PySimpleGUI

def take_picture():
    ##for desktop
    camera = cv2.VideoCapture(0) ##for desktop
    ret = camera.set(3,224)
    ret = camera.set(4,224)
    s, image = camera.read() ##for desktop
    camera.release()            
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
    image = PIL.Image.fromarray(image)
    
    #stream=BytesIO()  ##for rpi
    #camera.capture(stream,format='jpeg',resize=(224,224))  ##for rpi
    #stream.seek(0)  ##for rpi
    #image=PIL.Image.open(stream).convert('RGB')  ##for rpi

    return image

image = take_picture()

b = io.BytesIO()                   # open a binary, memory-resident file-like
#image.save(b, "ppm")                 # save the image to it using a tkinter compatible format
image.save(b, "png")
imdata = b.getvalue() 


layout = [  [sg.Text('Mushroom Type', size=(20, 1), font=('Helvetica', 12), justification='center')],
        [sg.Image(data = imdata, key='_IMAGE_')],
        [sg.Text('', size=(20, 1), font=('Helvetica', 12), justification='center', key='_CLASS_')],
        [sg.Text('', size=(20, 1), font=('Helvetica', 12), justification='center', key='_PROB_')],
        [sg.Button('Correction', focus=True), sg.Quit()]]

window = sg.Window('Mushroom AI', layout)

event, values = window.Read(timeout=1)

amplitude = .5
frequency = 1000
duration = 0.1
modulationf=1
start_idx = 0

def sinewave(t,amplitude = 0.5,frequency = 0,phase = 0):
    return amplitude*np.sin(phase+2 * np.pi * frequency * t)

def gate(t,duration):
    return (t<duration)

def periodicgate(t,frequency,delay = 0.25, stop=0.50):
    x=((t* frequency) % 1)/frequency
    return (x>=delay) & (x<=stop)

def sawtoothwave(t,frequency,delay=0):
    return (delay+  frequency * t) % 1 

#define pulse(x,duration):
    
#try:
samplerate = 16000

def callback(outdata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    global start_idx
    t = (start_idx + np.arange(frames)) / samplerate
    t = t.reshape(-1, 1)
    outdata[:] = sinewave(t,frequency=frequency) * periodicgate(t,modulationf,0,.05)
    start_idx += frames
    # use repeating function to repeat pulse (div or mod?).  Also can ramp up and down using modulating sine a lower freq




timer_running = True




#s, image = camera.read() ##for desktop
#image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
#plt.imshow(image)
#imgtk = PIL.ImageTk.PhotoImage(image)

while True:
    with sd.OutputStream( channels=1, callback=callback,
                         samplerate=samplerate):
        #print('#' * 80)
        #print('press Return to quit')
        #print('#' * 80)
        #frequency = frequency +100
        #sleep(.1)

        event, values = window.Read(timeout=1) # Please try and use as high of a timeout value as you can
        if event is None or event == 'Quit':    # if user closed the window using X or clicked Quit button
            break
        #elif event == 'Correction':
        #    timer_running = not timer_running
        if timer_running:



            image = take_picture()
            img=Image(pil2tensor(image,np.float32).div_(255))
            
            pred_class,pred_idx,outputs = learn.predict(img)
            sleep(.5)
            prob=outputs[0].tolist()
            #print(pred_class,prob)
            
            modulationf=prob*100
            b = io.BytesIO()                   # open a binary, memory-resident file-like
            image.save(b, "png")                 # save the image to it using a tkinter compatible format
            imdata = b.getvalue() 

            
            window.Element('_IMAGE_').Update(data=imdata )
            window.Element('_PROB_').Update(f'prob: {(prob*100.0):02.2f} %')
            window.Element('_CLASS_').Update(str(pred_class))
        
        





In [3]:
## uses a ram io file to store image to get to PySimpleGUI
## saves incorrect images

def take_picture():
    ##for desktop
    camera = cv2.VideoCapture(0) ##for desktop
    ret = camera.set(3,224)
    ret = camera.set(4,224)
    s, image = camera.read() ##for desktop
    camera.release()            
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
    image = PIL.Image.fromarray(image)
    
    #stream=BytesIO()  ##for rpi
    #camera.capture(stream,format='jpeg',resize=(224,224))  ##for rpi
    #stream.seek(0)  ##for rpi
    #image=PIL.Image.open(stream).convert('RGB')  ##for rpi

    return image

image = take_picture()

b = BytesIO()                   # open a binary, memory-resident file-like
image.save(b, "png")
imdata = b.getvalue() 


layout = [  [sg.Text('Mushroom Type', size=(20, 1), font=('Helvetica', 12), justification='center')],
        [sg.Image(data = imdata, key='_IMAGE_')],
        [sg.Text('', size=(20, 1), font=('Helvetica', 12), justification='center', key='_CLASS_')],
        [sg.Text('', size=(20, 1), font=('Helvetica', 12), justification='center', key='_PROB_')],
        [sg.Button('Correction', focus=True), sg.Quit()]]

window = sg.Window('Mushroom AI', layout,no_titlebar=True, location=(0,0), keep_on_top=True).Finalize()


window.Maximize()

event, values = window.Read(timeout=1)

amplitude = .5
frequency = 1000
duration = 0.1
modulationf=1
start_idx = 0

def sinewave(t,amplitude = 0.5,frequency = 0,phase = 0):
    return amplitude*np.sin(phase+2 * np.pi * frequency * t)

def gate(t,duration):
    return (t<duration)

def periodicgate(t,frequency,delay = 0.25, stop=0.50):
    x=((t* frequency) % 1)/frequency
    return (x>=delay) & (x<=stop)

def sawtoothwave(t,frequency,delay=0):
    return (delay+  frequency * t) % 1 

#define pulse(x,duration):
    
#try:
samplerate = 16000

def callback(outdata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    global start_idx
    t = (start_idx + np.arange(frames)) / samplerate
    t = t.reshape(-1, 1)
    outdata[:] = sinewave(t,frequency=frequency) * periodicgate(t,modulationf,0,.05)
    start_idx += frames
    # use repeating function to repeat pulse (div or mod?).  Also can ramp up and down using modulating sine a lower freq






#s, image = camera.read() ##for desktop
#image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  ##for desktop
#plt.imshow(image)
#imgtk = PIL.ImageTk.PhotoImage(image)

while True:
    with sd.OutputStream( channels=1, callback=callback,
                         samplerate=samplerate):
        #print('#' * 80)
        #print('press Return to quit')
        #print('#' * 80)
        #frequency = frequency +100
        #sleep(.1)

        event, values = window.Read(timeout=10) # Please try and use as high of a timeout value as you can
        if event is None or event == 'Quit':    # if user closed the window using X or clicked Quit button
            window.Close()
            break
            
            
        if event == 'Correction' and str(pred_class) == 'chanterelles':
            image.save(str(path/'corrections/other/not_chant') + strftime('%Y-%m-%d-%X')+str('.jpg'),'JPEG')
            
        if event == 'Correction' and str(pred_class) == 'other':
            image.save(str(path/'corrections/chanterelles/chant') + strftime('%Y-%m-%d-%X')+str('.jpg'),'JPEG')
        

        image = take_picture()
        img=Image(pil2tensor(image,np.float32).div_(255))

        pred_class,pred_idx,outputs = learn.predict(img)
        sleep(.5)
        prob=outputs[0].tolist()
        #print(pred_class,prob)

        modulationf=prob*100
        b = BytesIO()                   # open a binary, memory-resident file-like
        image.save(b, "png")                 # save the image to it using a tkinter compatible format
        imdata = b.getvalue() 


        window.Element('_IMAGE_').Update(data=imdata )
        window.Element('_PROB_').Update(f'prob: {(prob*100.0):02.2f} %')
        window.Element('_CLASS_').Update(str(pred_class))
        
        



In [ ]:
strftime('%Y-%m-%d-%X')